In [31]:
import pandas as pd

In [32]:
df = pd.read_csv(r"D:\Sem 8\PeopleBox\input.csv")

In [33]:
df

,Employee Code,Manager Employee Code,Date of Joining,Date of Exit,Compensation,Compensation 1,Compensation 1 date,Compensation 2,Compensation 2 date,Review 1,Review 1 date,Review 2,Review 2 date,Engagement 1,Engagement 1 date,Engagement 2,Engagement 2 date
0,1,NaN,2021-01-01,NaN,20000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,1.0,2021-01-01,NaN,20000,10000.0,2022-01-01,20000.0,2023-01-01,9.0,2021-06-01,9.5,2022-06-01,4.0,2021-03-01,5.0,2022-03-01
2,3,1.0,2021-01-01,2023-12-31,20000,10000.0,2022-01-01,20000.0,2023-01-01,9.0,2021-06-01,9.5,2022-06-01,4.0,2021-03-01,5.0,2022-03-01


### 1. Effective and End Dates

In [34]:
from datetime import datetime, timedelta

# Sort data by Employee Code and Date of Joining
df['Date of Joining'] = pd.to_datetime(df['Date of Joining'])
df.sort_values(by=['Employee Code', 'Date of Joining'], inplace=True)

# Initialize Effective Date and End Date lists
effective_dates = []
end_dates = []

# Iterate through each employee's records
for idx, row in df.iterrows():
    if idx == 0:
        # First record for the employee
        effective_date = row['Date of Joining']
    else:
        # One day after the previous record's End Date
        effective_date = end_dates[-1] + timedelta(days=1)

    effective_dates.append(effective_date)

    if idx == len(df) - 1:
        # Last record for the employee
        end_date = datetime(2100, 1, 1)
    else:
        # One day before the next record's Effective Date
        end_date = df.iloc[idx + 1]['Date of Joining'] - timedelta(days=1)

    end_dates.append(end_date)

# Append Effective Date and End Date columns to the DataFrame
df['Effective Date'] = effective_dates
df['End Date'] = end_dates

# Output the DataFrame with Effective Date and End Date appended
df


,Employee Code,Manager Employee Code,Date of Joining,Date of Exit,Compensation,Compensation 1,Compensation 1 date,Compensation 2,Compensation 2 date,Review 1,Review 1 date,Review 2,Review 2 date,Engagement 1,Engagement 1 date,Engagement 2,Engagement 2 date,Effective Date,End Date
0,1,NaN,2021-01-01,NaN,20000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-01-01,2020-12-31
1,2,1.0,2021-01-01,NaN,20000,10000.0,2022-01-01,20000.0,2023-01-01,9.0,2021-06-01,9.5,2022-06-01,4.0,2021-03-01,5.0,2022-03-01,2021-01-01,2020-12-31
2,3,1.0,2021-01-01,2023-12-31,20000,10000.0,2022-01-01,20000.0,2023-01-01,9.0,2021-06-01,9.5,2022-06-01,4.0,2021-03-01,5.0,2022-03-01,2021-01-01,2100-01-01


### 2. Data Transformation

In [48]:
import pandas as pd
from datetime import datetime, timedelta

# Function to handle missing values
def fill_missing_values(series):
    series = series.ffill()
    series.fillna(value=pd.NA, inplace=True)
    return series

# Fill missing values
for col in ['Compensation', 'Compensation 1', 'Compensation 2', 'Review 1', 'Review 2', 'Engagement 1', 'Engagement 2']:
    df[col] = fill_missing_values(df[col])

# Convert date columns to datetime
df['Date of Joining'] = pd.to_datetime(df['Date of Joining'])
df['Date of Exit'] = pd.to_datetime(df['Date of Exit'])
for col in ['Compensation 1 date', 'Compensation 2 date', 'Review 1 date', 'Review 2 date', 'Engagement 1 date', 'Engagement 2 date']:
    df[col] = pd.to_datetime(df[col])

# Initialize transformed data list
transformed_data = []

# Initialize dictionaries to keep track of most recent values
last_compensation = {}
last_review = {}
last_engagement = {}

# Iterate through each row
for idx, row in df.iterrows():
    employee_code = row['Employee Code']
    start_date = row['Date of Joining']
    end_date = row['Date of Exit'] if not pd.isnull(row['Date of Exit']) else datetime(2100, 1, 1)
    
    # Update last known values for each employee
    if not pd.isnull(row['Compensation']):
        last_compensation[employee_code] = row['Compensation']
    if not pd.isnull(row['Review 1']):
        last_review[employee_code] = row['Review 1']
    if not pd.isnull(row['Engagement 1']):
        last_engagement[employee_code] = row['Engagement 1']
    
    # Iterate through each compensation, review, and engagement record
    for i in range(1, 3):
        compensation = row[f'Compensation {i}']
        review = last_review.get(employee_code, pd.NA)
        engagement = last_engagement.get(employee_code, pd.NA)
        compensation_date = row[f'Compensation {i} date']
        review_date = row[f'Review {i} date']
        engagement_date = row[f'Engagement {i} date']
        
        # Append transformed data
        transformed_data.append({
            'Employee Code': employee_code,
            'Manager Employee Code': row['Manager Employee Code'],
            'Last Compensation': last_compensation.get(employee_code, pd.NA),
            'Compensation': compensation,
            'Last Pay Raise Date': last_compensation.get(employee_code, pd.NA),
            'Variable Pay': pd.NA,
            'Tenure in Org': (start_date - df.loc[0, 'Date of Joining']).days,
            'Performance Rating': review,
            'Engagement Score': engagement,
            'Effective Date': compensation_date,
            'End Date': review_date - timedelta(days=1) if not pd.isnull(review_date) else engagement_date - timedelta(days=1) if not pd.isnull(engagement_date) else end_date
        })

# Create DataFrame from transformed data
transformed_df = pd.DataFrame(transformed_data)
transformed_df


,Employee Code,Manager Employee Code,Last Compensation,Compensation,Last Pay Raise Date,Variable Pay,Tenure in Org,Performance Rating,Engagement Score,Effective Date,End Date
0,1,NaN,20000,NaN,20000,<NA>,0,<NA>,<NA>,NaT,2100-01-01
1,1,NaN,20000,NaN,20000,<NA>,0,<NA>,<NA>,NaT,2100-01-01
2,2,1.0,20000,10000.0,20000,<NA>,0,9.0,4.0,2022-01-01,2021-05-31
3,2,1.0,20000,20000.0,20000,<NA>,0,9.0,4.0,2023-01-01,2022-05-31
4,3,1.0,20000,10000.0,20000,<NA>,0,9.0,4.0,2022-01-01,2021-05-31
5,3,1.0,20000,20000.0,20000,<NA>,0,9.0,4.0,2023-01-01,2022-05-31


In [52]:
transformed_df

,Employee Code,Manager Employee Code,Last Compensation,Compensation,Last Pay Raise Date,Variable Pay,Tenure in Org,Performance Rating,Engagement Score,Effective Date,End Date
0,1,NaN,20000,NaN,20000,<NA>,0,<NA>,<NA>,NaT,2100-01-01
1,1,NaN,20000,NaN,20000,<NA>,0,<NA>,<NA>,NaT,2100-01-01
2,2,1.0,20000,10000.0,20000,<NA>,0,9.0,4.0,2022-01-01,2021-05-31
3,2,1.0,20000,20000.0,20000,<NA>,0,9.5,5.0,2023-01-01,2022-05-31
4,3,1.0,20000,10000.0,20000,<NA>,0,9.0,4.0,2022-01-01,2021-05-31
5,3,1.0,20000,20000.0,20000,<NA>,0,9.5,5.0,2023-01-01,2022-05-31


### 3.Output to CSV

In [53]:
transformed_df.to_csv('Output.csv', index=False)

In [54]:
cd

C:\Users\zeelt


C:\Users\zeelt\AppData\Local\Programs\Python\Python312\Lib\site-packages\IPython\core\magics\osm.py:393: UserWarning: using bookmarks requires you to install the `pickleshare` library.
  bkms = self.shell.db.get('bookmarks', {})
C:\Users\zeelt\AppData\Local\Programs\Python\Python312\Lib\site-packages\IPython\core\magics\osm.py:428: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]
